# Product Quantization

In [1]:
# English CoNLL17 corpus, Word2Vec Continuous Skipgram, 4027169 word embeddings 
# from http://vectors.nlpl.eu/repository/
f = open('data/word2vec/model.txt', encoding = "ISO-8859-1")
lines = f.readlines()

In [2]:
print(len(lines))
print(lines[0])
print(lines[3])


4027170
4027169 100

the -0.145301 -0.150899 0.389752 0.019979 0.209313 -0.059876 -0.037605 -0.191378 -0.492421 0.094606 0.471501 0.171619 -0.091895 -0.188857 -0.187415 0.102550 0.521365 0.492694 0.014640 0.324407 0.132155 0.392447 0.177873 -0.233233 -0.194923 -0.393363 0.018775 0.244165 -0.285453 -0.175422 -0.149823 -0.130606 -0.216950 0.308142 -0.192615 0.212360 0.090488 -0.229107 0.118502 0.217839 -0.379018 -0.042318 -0.315532 -0.186368 -0.028538 0.253762 0.487518 -0.055428 -0.239519 -0.209573 0.140636 -0.090901 -0.384449 0.447566 -0.184971 0.261921 0.440821 0.062585 0.181714 -0.252114 0.122724 0.015310 -0.143186 -0.209463 -0.174111 0.143348 0.295857 -0.156869 0.169965 0.038492 -0.122283 0.095772 0.314591 0.047793 -0.162416 -0.008667 -0.258904 0.129512 -0.086891 -0.131997 -0.256616 -0.071309 0.175144 0.060490 0.043357 0.224987 0.263239 0.236568 0.264060 0.109120 0.076898 0.172123 -0.248498 0.042334 0.053334 0.048210 0.239207 -0.083211 0.214255 -0.118595 



In [3]:
import numpy as np

keys = []
embeddings = []
for i in range(1, 100000): # use len(lines) for the whole table, testing on first 1000
    line = lines[i].split(" ")
    keys.append(line[0])
    embeddings.append(line[1:-1])

In [15]:
keys = []
embeddings = []
with open("data/word2vec/model.txt", encoding="ISO-8859-1") as infile:
    first = True
    for line in infile:
        if first:
            first = False
        else:
            vector = line.split(" ")
            keys.append(vector[0])
            embeddings.append(vector[1:-1])

In [16]:
print(keys[:50]) # first 50 keys

['</s>', ',', 'the', '.', 'of', 'and', 'to', 'a', 'in', '-', ')', '(', ':', 'for', 'is', '"', 'on', 'i', 'that', 'with', 'it', 'was', 'by', 'as', "'s", 'at', 'you', 'this', 'from', 'are', 'be', 'or', 'he', 'have', 'not', 'an', 'but', 'his', 'all', ';', 'your', 'they', '...', 'one', 'we', '?', 'has', 'more', 'new', 'will']


In [17]:
print(embeddings[-1]) # vector embedding example

['0.007251', '-0.075508', '0.200810', '-0.151546', '0.108970', '-0.026195', '0.067318', '-0.145598', '-0.003426', '0.180760', '0.115893', '0.109108', '-0.200582', '0.069990', '-0.079221', '0.190447', '0.142340', '0.012564', '0.070283', '0.098060', '-0.000247', '0.147612', '0.082730', '-0.191928', '0.046720', '-0.184546', '0.129292', '0.180053', '0.070508', '-0.018872', '0.010571', '0.167779', '-0.212089', '0.112918', '-0.108359', '-0.017143', '0.247880', '-0.303645', '0.149866', '-0.191488', '-0.094353', '0.094437', '0.009830', '0.006065', '0.035220', '0.108447', '0.084474', '-0.068517', '-0.031714', '0.002040', '0.041380', '0.058009', '-0.004007', '0.100894', '-0.005039', '-0.129871', '0.290422', '0.021951', '-0.100414', '-0.121083', '0.090845', '-0.065792', '-0.020135', '0.059742', '-0.042193', '0.046906', '0.157720', '-0.127840', '0.052213', '0.031162', '-0.001559', '-0.025602', '0.236345', '-0.147311', '0.032838', '0.235244', '-0.120139', '-0.032826', '-0.115913', '-0.199328', '0.1

In [4]:
import math
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def product_quantization(embeddings, M, k, verbose=False, silhouette_scoring=False, inertia_scoring=False):
    """
    keys: embeddings keys
    embeddings: embedding vectors
    M: size of vector subsections
    k: number of clusters for k-means clustering

    Runs product quantization on the embeddings.
    Returns each embedding as an array of integers, each representing the id of a centroid in that region
    """
    if verbose:
        print(embeddings)
    # build split embeddings
    # split_embeddings[i] -> list of vectors, each which represents the ith section of M units of embeddings
    num_subsections = math.ceil(len(embeddings[0]) / M)
    print(f"Splitting {len(embeddings)} embeddings of size {len(embeddings[0])} into {num_subsections} subsections of size {M}")
    split_embeddings = [[] for _ in range(num_subsections)]
    for embedding in embeddings:
        subsections = [embedding[i:i + M] for i in range(0, len(embedding), M)]
        for i in range(len(subsections)):
            split_embeddings[i].append(subsections[i])

    if verbose:
        print(split_embeddings)
    
    print(f"Performing k means search with k = {k}")
    embeddings_as_centroid_ids = [[] for _ in range(len(embeddings))]

    # given a subsection index and a centroid id within that subsection, return the centroid
    # codebooks = [
    #   [centroid0, centroid1, ..., centroidk], # subsection_0
    #   [centroid0, centroid1, ..., centroidk], # subsection_1
    #   ... 
    #   [centroid0, centroid1, ..., centroidk], # subsection_num_subsections
    # ]
    codebooks = [[] for _ in range(num_subsections)]
    section_index = 0
    for section in split_embeddings:
        X = np.array(section)
        if verbose:
            print("performing k means with ...")
            print(X)
        kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
        labels = kmeans.predict(X)
        centroids = kmeans.cluster_centers_
        if silhouette_scoring:
            return silhouette_score(X, kmeans.labels_, metric='euclidean')
        if inertia_scoring:
            return kmeans.inertia_
        for i in range(len(labels)):
            centroid_id = labels[i]
            embeddings_as_centroid_ids[i].append(centroid_id)
        for i in range(len(centroids)):
            codebooks[section_index].append(centroids[i])
        section_index += 1
    
    return embeddings_as_centroid_ids, codebooks

In [ ]:
# verified output from example here: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
# product_quantization([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]], 1, 2, verbose=True)

In [ ]:
# same example as above. with subsections of size M=2, centroids should be the same as above but repeated for the 2 subsections
# product_quantization([[1, 2, 1, 2], [1, 4, 1, 4], [1, 0, 1, 0], [10, 2, 10, 2], [10, 4, 10, 4], [10, 0, 10, 0]], 2, 2, verbose=True)

In [ ]:
# quantized, codebooks = product_quantization([[1, 2, 1, 2, 10, 2], [1, 4, 1, 4, 10, 4], [1, 0, 1, 0, 10, 0], [10, 2, 10, 2, 1, 2], [10, 4, 10, 4, 1, 4], [10, 0, 10, 0, 1, 0]], 3, 2, verbose=True)
# quantized, codebooks

# Theoretical Memory Use
* The size of the new centroid_id based embedding table should be N embeddings times (D / M) dimensional vectors times k for size of integers as opposed to N * D * 32
* The size of the codebooks should be 


# Silhouette Method and Elbow Method to determine optimal k for k-Means

Very low silhouette scores and the lack of abnormal drops in the inertia curve signals to me that there is no single "best" value of k for k means. Given these observations, choosing k=256 seems to be the most moderate choice in balancing embedding variance and compression. K=256 also seems to be a good choice for the "elbow" of the inertia curve. 

In [11]:
# import random
# sampled_embeddings = random.sample(embeddings, 10000)
# silhouette_scores = []
# for k in [16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192]:
#     silhouette_scores.append(product_quantization(sampled_embeddings, M=10, k=k, silhouette_scoring=True))
# silhouette_scores
# [0.07371708125729268,
#  0.06497374546551112,
#  0.06076052094252413,
#  0.06372546359032381,
#  0.06776646181674541,
#  0.07199628263885234,
#  0.07976291826725852,
#  0.08442804342327115,
#  0.0766254825841382,
#  0.04187258368671491]

In [12]:
# distortion_scores = []
# for k in [16, 32, 64, 90, 128, 180, 256, 375, 512, 1024, 2048, 4096]:
#    distortion_scores.append(product_quantization(sampled_embeddings, M=10, k=k, inertia_scoring=True))
# distortion_scores

In [13]:
# import matplotlib.pyplot as plt
# plt.scatter([16, 32, 64, 90, 128, 180, 256, 375, 512, 1024, 2048, 4096], distortion_scores, marker='o');

In [ ]:
quantized, codebooks = product_quantization(embeddings, M=10, k=256)

In [176]:
np.savetxt('data/word2vec/keys.txt', np.array(keys), fmt="%s")
np.savetxt('data/word2vec/quantized.txt', quantized, fmt='%i')
np.savetxt('data/word2vec/codebooks.txt', np.array(codebooks).reshape(np.array(codebooks).shape[0], -1), fmt='%i')
print(len(codebooks[0][0]))

3
